# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install numpy==1.26.1

In [3]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [4]:
import wandb # Logging
from google.colab import userdata
my_secret = userdata.get("wandb_api_key")

wandb.login(key=my_secret)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shoann (shoann-mycompany-inc) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
from unsloth import FastLanguageModel # model library
import torch # NN library
from datasets import load_dataset # hf dataset loader

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# Load Model

In [6]:
model_id = "unsloth/Qwen3-8B-unsloth-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

==((====))==  Unsloth 2025.5.1: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/144k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.5.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


# Load Data

In [8]:
dataset = load_dataset("ShoAnn/legalqa_klinik_hukumonline")
train_valid = dataset["train"].train_test_split(test_size=0.1, seed=42)

from datasets import DatasetDict
# Gather the splits into a DatasetDict
dataset = DatasetDict({
    "train": train_valid["train"],
    "valid": train_valid["test"]
})

README.md:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.62M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/547k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1006 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

In [9]:
# from unsloth.chat_templates import get_chat_template
# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "gemma-3",
# )

In [10]:
instruction_prompt = "You are an AI Legal Assistant. Your task is to carefully analyze the provided **Legal Basis** below and answer given **Question** based *solely* and *exclusively* on the information contained within that context in Indonesian Language."
def format_to_conv(examples):
    questions = examples["question"]
    inputs       = examples["context"]
    outputs      = examples["answer"]
    conversation = []
    for question, input, output in zip(questions, inputs, outputs):
        conversation.append([ # This line has been changed
            {'content': f'{instruction_prompt} \n **Question** \n{question} \n**Legal Basis** \n{input}', 'role': 'user'},
            {'content': output, 'role': 'assistant'}]
        )
    return { "conversations" : conversation, }

train_dataset = dataset['train'].map(format_to_conv, batched = True,)
valid_dataset = dataset['valid'].map(format_to_conv, batched = True,)

Map:   0%|          | 0/905 [00:00<?, ? examples/s]

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

In [11]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

train = train_dataset.map(formatting_prompts_func, batched = True)
eval = valid_dataset.map(formatting_prompts_func, batched = True) # eval training AKA validation

Map:   0%|          | 0/905 [00:00<?, ? examples/s]

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

# Train Model

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

wandb.init(
    # track hyperparameters and run metadata
    config={
    "model": "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "dataset": "ShoAnn/legalqa_klinik_hukumonline",
    "per_device_train_batch_size": 4,
    "gradient_accumulation_steps": 8,
    "warmup_steps": 5,
    "num_train_epochs": 10,
    "learning_rate": 2e-4,
    "fp16": "not is_bfloat16_supported()",
    "bf16": "is_bfloat16_supported()",
    "logging_steps": 1,
    "optim": "adamw_8bit",
    "weight_decay": 0.01,
    "lr_scheduler_type": "linear",
    "seed": 3407,
    },
    entity="shoann-mycompany-inc",
    project=model_id.split("/")[1],
    # id="",
    resume="allow"
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train,
    eval_dataset = eval,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        run_name = "gemma3-4b",
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        num_train_epochs = 10, # Set this for 1 full training run.
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "/content/drive/MyDrive/model_outputs/qwen3/",
        save_strategy = "steps",
        save_total_limit = 2,
        save_steps = 10,
        report_to = "wandb", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/905 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/101 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 905 | Num Epochs = 10 | Total steps = 280
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 87,293,952/8,000,000,000 (1.09% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.405700
2,1.402800
3,1.408000
4,1.568400
5,1.463800
6,1.418000
7,1.413700
8,1.372700
9,1.445400
10,1.378000


In [ ]:
trainer.push_to_hub(f"ShoAnn/Qwen3-8B-unsloth-bnb-4bit", tokenizer, token = userdata.get('HF_TOKEN'))

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")